In [ ]:
import torch
import numpy as np
import pandas as pd
from qpsolvers import solve_qp
from scipy.optimize import linprog

In [13]:
! pip install daqp

Defaulting to user installation because normal site-packages is not writeable
  Using cached daqp-0.5.1.tar.gz (132 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for daqp (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Could not find daqp directory
      running bdist_wheel
      running build
      running build_ext
      building 'daqp' extension
      error: unknown file type '.pxd' (from 'daqp.pxd')
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for daqp
Failed to build daqp
ERROR: Could not build wheels for daqp, which is required to install pyproject.toml-based projects


In [ ]:
! pip3 install qpsolvers

In [11]:
! conda install -c conda-forge qpsolvers -y

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:
/ 
  - conda-forge/linux-ppc64le::gcc_impl_linux-ppc64le==12.2.0=h3a350b8_18
  - conda-forge/noarch::sysroot_linux-ppc64le==2.17=h395ec9b_13
  - conda-forge/linux-ppc64le::clang==15.0.2=ha3edaa6_0
  - conda-forge/linux-ppc64le::binutils_impl_linux-ppc64le==2.39=h6c0ba3f_0
failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/linux-ppc64le::shap==0.40.0=py39h92fcf75_0
  - conda-forge/linux-ppc64le::gcc_impl_linux-ppc64le==12.2.0=h3a350b8_18
  - conda-forge/noarch::sysroot_linux-ppc64le==2.17=h395ec9b_13
  - conda-forge/linux-ppc64le::clang==15.0.2=ha3edaa6_0

In [ ]:
df = pd.read_csv("../data/student/student-mat.csv", sep=';')

In [ ]:
df_encoded = pd.get_dummies(df, columns=["school", "sex", "address", "famsize", "Pstatus", "Mjob", "Fjob", "reason", "guardian", 
                                         "schoolsup", "famsup", "paid", "activities", "nursery", "higher", "internet", "romantic"], drop_first=True)
df_encoded.drop(["G1", "G2"], inplace=True, axis=1)

In [ ]:
features = df_encoded.drop(["G3"], axis=1)
target = df_encoded["G3"]
X = torch.tensor(features.values, dtype=torch.float)
X = X - X.mean(dim=0)
X = X/X.std(dim=0)
Y = torch.tensor(target.values, dtype=torch.float)
Y = Y - Y.mean()

In [9]:
class ContSolve(torch.nn.Module):
    def __init__(self, X, Y, k, l, device):
        super(ContSolve, self).__init__()
        self.X = X.to(torch.device(device))
        self.Y = Y.to(torch.device(device))
        for tensor in [self.X, self.Y]:
            tensor.requires_grad = False
        self.m = self.X.shape[1]
        self.n = self.X.shape[0]
        self.k = k
        self.l = l
        self.device = torch.device(device)
        self.ps = torch.nn.Parameter(torch.full((self.m,), self.k/self.m).to(self.device), requires_grad=True)
        self.qs = torch.nn.Parameter(torch.full((self.n,), self.l/self.n).to(self.device), requires_grad=True)
        
#     def forward2(self):
#         tilde_M = torch.einsum('m,mn,n->mn',self.ps,self.M,self.ps)
#         init_power = torch.eye(self.m).to(self.device)-tilde_M
#         weight_X = torch.einsum('m,mn->mn',self.ps,self.X.T)
#         with torch.no_grad():
#             eps_0 = torch.norm(weight_X)
#             alpha = torch.norm(init_power)
#             print((eps_0, alpha))
#             print(torch.round(torch.log(self.eps*(1-alpha)/eps_0)/torch.log(alpha)+0.5))
#             k = torch.max(1, torch.round(torch.log(self.eps*(1-alpha)/eps_0)/torch.log(alpha)+0.5))
#         total = torch.eye(self.m).to(self.device)
#         curr_power = torch.eye(self.m).to(self.device)
#         for i in range(1, k+1):
#             curr_power = torch.einsum('mn,np->mp',curr_power,init_power)
#             total += curr_power
#         tilde_X_terms = torch.einsum('mp,pn->mn',total,weight_X)
#         beta = torch.einsum('mn,n->m',tilde_X_terms,self.Y) 
#         resid = self.Y - torch.einsum('nm,m->n',weight_X,beta)
#         r2 = 1 - torch.einsum('n,n->1',resid,resid)/torch.einsum('n,n->1',self.Y,self.Y)
#         return r2
    
    def forward(self):
        scaled_X = torch.einsum('n,nm,m->nm',self.qs,self.X,self.ps)
        scaled_Y = torch.einsum('n,n->n',self.Y,self.qs)
        tilde_M = torch.einsum('mn,np->mp',scaled_X.T,scaled_X)
        tilde_M_inv = torch.linalg.inv(tilde_M)
        tilde_M_inv_Xt = torch.einsum('mn,np->mp',tilde_M_inv,scaled_X.T)
        beta = torch.einsum('mn,n->m',tilde_M_inv_Xt,scaled_Y) 
        resid = scaled_Y - torch.einsum('nm,m->n',scaled_X,beta)
        r2 = 1 - torch.einsum('n,n->',resid,resid)/torch.einsum('n,n->',scaled_Y,scaled_Y)
        return r2
    
def project_onto_simplex(v, select, total):
    P = 2*np.eye(total)
    q = 2*v
    A = np.ones((1, total))
    b = np.array([select])
    lb = np.zeros(total)
    ub = np.ones(total)
    proj = solve_qp(P, q, A=A, b=b, lb=lb, ub=ub)
    return proj
    
def frank_wolfe(gradV, select, total):
    c = -gradV
    A_eq = np.ones((1, total))
    b_eq = np.array([select])
    bounds = (0, 1)
    res = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds)
    return res

In [191]:
m = X.shape[1]
n = X.shape[0]
k = 10
l = 40
cs = ContSolve(X, Y, k, l, "cuda")

In [192]:
learning_rate = 10
for epoch in range(1, 11):
    # backprop on rows
    cs.qs.requires_grad = True
    cs.ps.requires_grad = False
    r2 = cs()
    print(r2)
    loss = r2
    loss.backward() 
    with torch.no_grad():
        cs.qs -= learning_rate * cs.qs.grad
    proj = project_onto_simplex(cs.qs.cpu().numpy(), l, n)
    cs.qs.data = torch.Tensor(proj).to("cuda")
    # backprop on cols
    cs.qs.requires_grad = False
    cs.ps.requires_grad = True
    r2 = cs()
    print(r2)
    loss = -r2
    loss.backward() 
    with torch.no_grad():
        cs.ps -= learning_rate * cs.ps.grad
    proj = project_onto_simplex(cs.ps.cpu().numpy(), k, m)
    cs.ps.data = torch.Tensor(proj).to("cuda")

tensor(0.9926, device='cuda:0', grad_fn=<RsubBackward1>)
Parameter containing:
tensor([0.1060, 0.1043, 0.1031, 0.1043, 0.1013, 0.1013, 0.1015, 0.1041, 0.1062,
        0.1023, 0.1030, 0.1013, 0.1013, 0.1015, 0.1040, 0.1031, 0.1017, 0.1018,
        0.1018, 0.1040, 0.1014, 0.1013, 0.1023, 0.1013, 0.1028, 0.1013, 0.1013,
        0.1013, 0.1013, 0.1013, 0.1031, 0.1039, 0.1091, 0.1020, 0.1047, 0.1049,
        0.1087, 0.1030, 0.1013, 0.1042, 0.1030, 0.1013, 0.1052, 0.1016, 0.1014,
        0.1046, 0.1013, 0.1055, 0.1040, 0.1045, 0.1020, 0.1013, 0.1045, 0.1022,
        0.1013, 0.1013, 0.1018, 0.1044, 0.1018, 0.1082, 0.1017, 0.1079, 0.1017,
        0.1013, 0.1017, 0.1061, 0.1013, 0.1076, 0.1029, 0.1025, 0.1021, 0.1044,
        0.1021, 0.1025, 0.1014, 0.1016, 0.1042, 0.1013, 0.1124, 0.1092, 0.1038,
        0.1018, 0.1123, 0.1014, 0.1013, 0.1013, 0.1051, 0.1016, 0.1032, 0.1103,
        0.1021, 0.1075, 0.1037, 0.1013, 0.1013, 0.1031, 0.1077, 0.1039, 0.1030,
        0.1017, 0.1111, 0.1032, 0.1022, 0

In [ ]:
for epoch in range(1, 11):
    t_k = 2/(epoch+2)
    # backprop on rows
    cs.qs.requires_grad = True
    cs.ps.requires_grad = False
    r2 = cs()
    print(r2)
    loss = r2
    loss.backward() 
    with torch.no_grad():
        y = frank_wolfe(cs.qs.grad, l, n)
    cs.qs.data = (1-t_k) * cs.qs.data + t_k * torch.Tensor(y).to("cuda")
    # backprop on cols
    cs.qs.requires_grad = False
    cs.ps.requires_grad = True
    r2 = cs()
    print(r2)
    loss = -r2
    loss.backward() 
    with torch.no_grad():
        y = frank_wolfe(cs.ps.grad, k, m)
    t_k = 2/(epoch+2)
    cs.ps.data = (1-t_k) * cs.ps.data + t_k * torch.Tensor(y).to("cuda")

In [97]:
np.where(cs.qs.data.cpu().numpy() == 1)

(array([252, 269, 276, 311, 349, 350, 351, 353, 354, 355, 356, 357, 358,
        359, 360, 362, 363, 364, 365, 366, 367, 368, 369, 370, 372, 373,
        374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386,
        387, 388, 389, 390, 391, 392, 393, 394]),)

In [89]:
def validate(ps, qs):
    
    cols = features.columns[(ps == 1).cpu().numpy()]
    features[cols]


KeyError: "['G3'] not found in axis"